In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,32813
2,Huelva,Confirmados PDIA 14 días,1237
3,Huelva,Tasa PDIA 14 días,"241,0507239316406"
4,Huelva,Confirmados PDIA 7 días,515
5,Huelva,Total Confirmados,32995
6,Huelva,Curados,30689
7,Huelva,Fallecidos,388


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  32813.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8948.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 741 personas en los últimos 7 días 

Un positivo PCR cada 272 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,32813.0,515.0,1237.0,513170.0,100.356607,241.050724,81.0
Huelva-Costa,18902.0,281.0,732.0,289548.0,97.047812,252.807825,37.0
Condado-Campiña,10654.0,178.0,389.0,156231.0,113.933854,248.990277,34.0
Huelva (capital),8948.0,194.0,528.0,143837.0,134.874893,367.082183,21.0
Almonte,1897.0,36.0,57.0,24507.0,146.896805,232.586608,9.0
Sierra de Huelva-Andévalo Central,2905.0,54.0,112.0,67391.0,80.129394,166.194299,9.0
Valverde del Camino,651.0,30.0,62.0,12750.0,235.294118,486.274510,6.0
Rociana del Condado,569.0,11.0,12.0,7939.0,138.556493,151.152538,6.0
Moguer,1366.0,28.0,70.0,21867.0,128.046829,320.117071,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,13.0,1.0,5.0,299.0,334.448161,1672.240803,0.0
Puebla de Guzmán,139.0,10.0,19.0,3092.0,323.415265,614.489004,3.0
San Juan del Puerto,560.0,22.0,48.0,9411.0,233.768994,510.041441,1.0
Valverde del Camino,651.0,30.0,62.0,12750.0,235.294118,486.274510,6.0
Huelva (capital),8948.0,194.0,528.0,143837.0,134.874893,367.082183,21.0
Almendro (El),39.0,2.0,3.0,840.0,238.095238,357.142857,0.0
Lucena del Puerto,265.0,6.0,11.0,3261.0,183.992640,337.319841,1.0
Calañas,200.0,8.0,9.0,2768.0,289.017341,325.144509,0.0
Moguer,1366.0,28.0,70.0,21867.0,128.046829,320.117071,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Punta Umbría,841.0,4.0,28.0,15355.0,26.050147,182.351026,0.0,0.142857
Cortelazor,13.0,1.0,5.0,299.0,334.448161,1672.240803,0.0,0.200000
Aljaraque,1157.0,10.0,45.0,21474.0,46.567943,209.555742,0.0,0.222222
Trigueros,329.0,6.0,22.0,7862.0,76.316459,279.827016,0.0,0.272727
Bonares,291.0,5.0,18.0,6060.0,82.508251,297.029703,0.0,0.277778
Gibraleón,645.0,4.0,13.0,12737.0,31.404569,102.064850,2.0,0.307692
Huelva (capital),8948.0,194.0,528.0,143837.0,134.874893,367.082183,21.0,0.367424
Bollullos Par del Condado,1025.0,16.0,43.0,14387.0,111.211510,298.880934,2.0,0.372093
Huelva-Costa,18902.0,281.0,732.0,289548.0,97.047812,252.807825,37.0,0.383880
